In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import nltk
%matplotlib inline

from bs4 import BeautifulSoup
import requests
import re
import pyinputplus as pyip
import requests
import datetime as dt
from datetime import datetime
from tqdm import tqdm
import pickle
import random
import math
import string

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 150)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Import other files
%run climbconstants.py

In [14]:
picklefile = open('../Data_Archive/df_archive_base', 'rb')
df_usend_uniq = pickle.load(picklefile)

In [ ]:
def upload_df(upload_type, link):
    try:
        df = pd.read_csv(f'{link}/{upload_type}-export')
    except Exception as err:
        print(err)
    return df

In [2]:
def data_standardize(df_source):
    ### Standardizes and slims dataset to only include relevant data entries. In particular the "Route Type" column which designates what kind of downstream data treatment the entry receives.
    ### One of the major
    
    # Some cleanup is only necessary on the ticked list, and not the todo list.
    if 'Date' in df_source.columns:
        df_source['Date'] = pd.to_datetime(df_source['Date']) # 'Date' to datetype
        df_source.rename(columns={'Pitches': 'Pitches Ticked'}, inplace=True) # Pitches relabeled to Pitches Ticked
        df_source['Notes'] = df_source['Notes'].apply(lambda x: str(x).replace('&#39;',"'")) # Apostrophe's are html coded in the notes section for some reason.

    # Remove all aid, ice, snow, TR only,and trad/boulder climbing route types as they are not relevant.
    df_source = df_source[df_source['Route Type'].str.contains(r'Aid|Ice|Snow') != True]
    df_source = df_source[df_source['Route Type'].str.fullmatch(r'TR') != True] #if this is just a partial match it will detech "trad" too!
    df_source = df_source[df_source['Route Type'].str.contains(r'Trad') & df_source['Route Type'].str.contains(r'Boulder') != True]

    # "trad, sport" goes to "trad". If it uses gear it's trad!
    df_source.loc[df_source['Route Type'].str.contains(r'Trad') & df_source['Route Type'].str.contains(r'Sport'), 'Route Type'] = 'Trad'

    # "x, alpine" and "x, tr" goes to "x" Alpine and tr tags are not useful.
    def rem_route_el_from_list(ousted, seperator):
        el_rem_subset = df_source['Route Type'].str.contains(ousted) == True
        df_source.loc[el_rem_subset, 'Route Type'] = df_source[el_rem_subset]['Route Type'].apply(lambda row: [val for val in row.split(seperator) if val != ousted]).apply(lambda x: ", ".join(x))

    rem_route_el_from_list('Alpine', ', ')
    rem_route_el_from_list('TR', ', ')

    # Extract route unique identifier from URL and create a new column for it.
    if 'Route ID' not in df_source.columns:
        df_source.insert(len(df_source.columns),'Route ID','')
    df_source['Route ID'] = df_source['URL'].apply(lambda x: x.split('/')[4])

    # Change YDS-Vgrade combos to just Vgrade. They are most likely boulders, so a bouldering grade is relevant.
    subset = df_source['Rating'].apply(lambda row: [val for val in row.split() if val in V_GRADES_FULL]).astype(bool)  & df_source['Rating'].apply(lambda row: [val for val in row.split() if val in YDS_GRADES_FULL]).astype(bool) == True
    df_source.loc[subset, 'Rating'] = df_source[subset]['Rating'].apply(lambda x: x.split()[1])

    # Seperate risk rating to new column
    if 'Rating' not in df_source.columns:
        df_source.insert(df_source.columns.get_loc('Rating')+1,'Risk','')
    df_source['Risk'] = df_source['Rating'].apply(lambda row: [val for val in row.split() if val in RISK_GRADES]).apply(lambda x: "".join(x))
    # Reduce Rating column to just rating
    df_source['Rating'] = df_source['Rating'].apply(lambda row: [val for val in row.split()][0])
    # Displays all rows where risk ratings have been seperated
    # df_usend[df_usend['Risk'].astype(bool)]

    # Create original rating and length archive to compare against or undo changes.
    if 'Original Rating' not in df_source.columns:
        df_source.insert(df_source.columns.get_loc('Rating'),'Original Rating',df_source['Rating'])

    return df_source

In [3]:
def route_length_fixer(df_source):
### Handle route length outliers and NaNs
    # Define roped and bouldering specific subset
    roped_subset = (df_source['Route Type'] == 'Sport') | (df_source['Route Type'] == 'Trad')
    boulder_subset = (df_source['Route Type'] == 'Boulder')

    ROPED_MIN_LENGTH = 15
    ROPED_MAX_LENGTH = 4500 #"Trango Towers" are 4,300' tall
    ROPED_DEFAULT_LENGTH = 70
    BOULDER_MIN_LENGTH = 1
    BOULDER_MAX_LENGTH = 55 #"Too Tall to Fall" is 50'
    BOULDER_DEFAULT_LENGTH = 12

    # Fix outliers
    def fix_length_outliers(dataframe, subset, minlength, maxlength, deflength):
        length_outliers = dataframe[subset][(dataframe[subset]['Length'] <= minlength) | (dataframe[subset]['Length'] >= maxlength)]
        for loop_count, (index, data) in enumerate(length_outliers.iterrows()):
            if skipchoice == 'y':
                updated_length = deflength
            else:
                updated_length = pyip.inputNum(f"[{loop_count+1}/{length_outliers.shape[0]}] Outlier Detected, Possible Bad Info. Input Corrected Length for\nRoute: {data['Route']}\nLocation: {'>'.join(data['Location'].split('>')[-3:])}\nCurrently: {data['Length']}ft\n", min=minlength, max=maxlength)
            dataframe.at[index, 'Length'] = updated_length
        return dataframe

    # Fill empty route lengths
    def fill_length_empties(dataframe, subset, minlength, maxlength, deflength):
        length_missing = dataframe[subset][(dataframe[subset]['Length'].isnull()) | (dataframe[subset]['Length'] == 0)]
        for loop_count, (index, data) in enumerate(length_missing.iterrows()):
            if skipchoice == 'y':
                updated_length = deflength
            else:
                updated_length = pyip.inputNum(f"[{loop_count+1}/{length_missing.shape[0]}] Input Estimated Length for\nRoute: {data['Route']}\nLocation: {'>'.join(data['Location'].split('>')[-3:])}\n", min=minlength, max=maxlength)
            dataframe.at[index, 'Length'] = updated_length
        return dataframe

    print(F'Would you like to skip manual length assignment and assign a common value of {ROPED_DEFAULT_LENGTH}ft for routes and {BOULDER_DEFAULT_LENGTH}ft for boulders to missing values?')
    skipchoice = pyip.inputChoice(['y','n'])
    df_source = fix_length_outliers(df_source, roped_subset, ROPED_MIN_LENGTH, ROPED_MAX_LENGTH, ROPED_DEFAULT_LENGTH)
    df_source = fill_length_empties(df_source, roped_subset, ROPED_MIN_LENGTH, ROPED_MAX_LENGTH, ROPED_DEFAULT_LENGTH)
    df_source = fix_length_outliers(df_source, boulder_subset, BOULDER_MIN_LENGTH, BOULDER_MAX_LENGTH, BOULDER_DEFAULT_LENGTH)
    df_source = fill_length_empties(df_source, boulder_subset, BOULDER_MIN_LENGTH, BOULDER_MAX_LENGTH, BOULDER_DEFAULT_LENGTH)

    return df_source

In [4]:
def grade_homo(df_source):
### Homogenize Grades
    grade_homo_choice = ['round down', 'round up', 'round evenly', 'hand determine']
    rating_isolate = df_source['Original Rating'].apply(lambda row: [val for val in row.split()][0])

    # Reset 'Rating' column so this mapping can be re-run
    df_source["Rating"] = df_source["Original Rating"]

    #Roped Grades
    print('ROPED GRADES')
    print('Select Roped Grade Homogenization Type:\n1)5.10a / 5.10b / 5.10c / 5.10d (Requires additional effort)\n2)5.10- / 5.10 / 5.10+')
    r_grade_homo_type = pyip.inputChoice(['1','2'])
    if r_grade_homo_type == '1':
        r_grade_homo_dir = pyip.inputChoice(grade_homo_choice)


    def grademoderate():
        grade_change_subset = rating_isolate.isin(list(rgrademoderatemap.keys()))
        df_source.loc[grade_change_subset, 'Rating'] = df_source.loc[grade_change_subset]['Original Rating'].map(rgrademoderatemap)

    def grade_split(upmap, downmap):
        grade_change_subset = rating_isolate.isin(list(upmap.keys()))
        grade_change_subset_df = df_source[grade_change_subset]
        for grade in grade_change_subset_df['Original Rating'].unique():
            to_change = grade_change_subset_df[grade_change_subset_df['Original Rating'] == grade]
            changed_up = to_change.sample(frac=0.5)['Original Rating'].map(upmap)
            df_source.loc[changed_up.index, 'Rating'] = changed_up
        grade_change_subset = rating_isolate.isin(list(downmap.keys()))
        grade_change_subset_df = df_source[grade_change_subset]
        for grade in grade_change_subset_df['Original Rating'].unique():
            to_change = grade_change_subset_df[grade_change_subset_df['Original Rating'] == grade]
            changed_down = to_change['Original Rating'].map(downmap)
            df_source.loc[changed_down.index, 'Rating'] = changed_down

    if r_grade_homo_type == '2':
        grade_change_subset = rating_isolate.isin(list(rgradecompmap.keys()))
        df_source.loc[grade_change_subset, 'Rating'] = df_source[grade_change_subset]['Original Rating'].map(rgradecompmap)
    else:
        if r_grade_homo_dir == grade_homo_choice[0]:
            grademoderate()
            grade_change_subset = rating_isolate.isin(list(rgradedownmap.keys()))
            df_source.loc[grade_change_subset, 'Rating'] = df_source[grade_change_subset]['Original Rating'].map(rgradedownmap)
        if r_grade_homo_dir == grade_homo_choice[1]:
            grademoderate()
            grade_change_subset = rating_isolate.isin(list(rgradeupmap.keys()))
            df_source.loc[grade_change_subset, 'Rating'] = df_source[grade_change_subset]['Original Rating'].map(rgradeupmap)
        if r_grade_homo_dir == grade_homo_choice[2]:
            grademoderate()
            grade_split(rgradeupmap,rgradedownmap)
        if r_grade_homo_dir == grade_homo_choice[3]:
            needs_grade_corr = df_source[rating_isolate.isin(list(rgrademoderatemap.keys()) + list(rgradedownmap.keys()))]
            for loop_count, (index, data) in enumerate(needs_grade_corr.iterrows()):
                updated_grade = pyip.inputChoice(prompt=f"[{loop_count+1}/{needs_grade_corr.shape[0]}] Input Grade Correction For: {data['Route'].title()}:\n", choices=YDS_GRADES_LETTER)
                df_source.at[index, 'Rating'] = updated_grade

    #Boulder Grades
    print('BOULDER GRADES')
    print('Select Boulder Grade Homogenization Type:\n1)V1\n2)V1- / V1 / V1+')
    b_grade_homo_type = pyip.inputChoice(['1','2'])
    b_grade_homo_dir = pyip.inputChoice(grade_homo_choice)


    if b_grade_homo_type == '1':
        # Remove all + and - grades
        grade_change_subset = rating_isolate.isin(list(bgradeconmap1.keys()))
        df_source.loc[grade_change_subset, 'Rating'] = df_source[grade_change_subset]['Original Rating'].map(bgradeconmap1)

        if b_grade_homo_dir == grade_homo_choice[0]:
            grade_change_subset = rating_isolate.isin(list(bgradedownmap1.keys()))
            df_source.loc[grade_change_subset, 'Rating'] = df_source[grade_change_subset]['Original Rating'].map(bgradedownmap1)
        if b_grade_homo_dir == grade_homo_choice[1]:
            grade_change_subset = rating_isolate.isin(list(bgradeupmap1.keys()))
            df_source.loc[grade_change_subset, 'Rating'] = df_source[grade_change_subset]['Original Rating'].map(bgradeupmap1)
        if b_grade_homo_dir == grade_homo_choice[2]:
            grade_split(bgradeupmap1,bgradedownmap1)
        if b_grade_homo_dir == grade_homo_choice[3]:
            needs_grade_corr = df_source[rating_isolate.isin(list(bgradedownmap1.keys()))]
            for loop_count, (index, data) in enumerate(needs_grade_corr.iterrows()):
                updated_grade = pyip.inputChoice(prompt=f"[{loop_count+1}/{needs_grade_corr.shape[0]}] Input Grade Correction For: {data['Route'].title()}:\n", choices=V_GRADES_BASIC)
                df_source.at[index, 'Rating'] = updated_grade

    if b_grade_homo_type =='2':
        if b_grade_homo_dir == grade_homo_choice[0]:
            grade_change_subset = rating_isolate.isin(list(bgradedownmap2.keys()))
            df_source.loc[grade_change_subset, 'Rating'] = df_source[grade_change_subset]['Original Rating'].map(bgradedownmap2)
        if b_grade_homo_dir == grade_homo_choice[1]:
            grade_change_subset = rating_isolate.isin(list(bgradeupmap2.keys()))
            df_source.loc[grade_change_subset, 'Rating'] = df_source[grade_change_subset]['Original Rating'].map(bgradeupmap2)
        if b_grade_homo_dir == grade_homo_choice[2]:
            grade_split(bgradeupmap2,bgradedownmap2)
        if b_grade_homo_dir == grade_homo_choice[3]:
            needs_grade_corr = df_source[rating_isolate.isin(list(bgradedownmap2.keys()))]
            for loop_count, (index, data) in enumerate(needs_grade_corr.iterrows()):
                updated_grade = pyip.inputChoice(prompt=f"[{loop_count+1}/{needs_grade_corr.shape[0]}] Input Grade Correction For: {data['Route'].title()}:\n", choices=V_GRADES_BASIC)
                df_source.at[index, 'Rating'] = updated_grade
    
    grade_settings = {'r_grade_homo_type': r_grade_homo_type, 'r_grade_homo_dir': r_grade_homo_dir, 'b_grade_homo_type': b_grade_homo_type, 'b_grade_homo_dir': b_grade_homo_dir}
    
    return df_source, grade_settings

In [ ]:
def user_uniq_clean(df_source):
    ### Cleans up uniq dataframe of userdataframe specific columns that are not relevant.
    col_list = ['Date', 'Notes', 'Your Stars', 'Style', 'Lead Style']
    for col in col_list:
        if col in df_source.columns:
            df_source.drop(columns=col, inplace=True)
    return df_source

In [5]:
def route_scrape(df_source):
    ### Downloads the route page and stat page for each entry. It is suggested you pass this a list of unique routes so it does not download redundantly.
    
    res_list = []
    if 'Re mainpage' not in df_source.columns:
        df_source.insert(len(df_source.columns),'Re mainpage','')
    if 'Re statpage' not in df_source.columns:
        df_source.insert(len(df_source.columns),'Re statpage','')
    
    def insert_str_to_address(url, insert_phrase):
        str_list = url.split('/')
        str_list.insert(4, insert_phrase)
        return '/'.join(str_list)

    def page_download(url):
        try:
            res = requests.get(url, timeout=10)
        except Exception as e:
            print(e)
            res = ''
        res_list.append(res) # Can check this list for errors
        
        return res

    df_source['Re Mainpage'] = df_source['URL'].progress_apply(page_download)
    df_source['Re Statpage'] = df_source['URL'].progress_apply(lambda x: page_download(insert_str_to_address(x, 'stats')))

    return(df_source)

In [10]:
def extract_default_pitch(df_source):
    ### Analyze the mainpage for listed default pitch lengths. Necessary for a user's tick export as it includes pitches as their own recorded pitch count. Not required for ToDo exports as it correctly lists the "official" pitch count.

    def get_pitches(res):
        soup = BeautifulSoup(res.text, 'html.parser')
        route_type_text = str(soup.find(class_="description-details").find_all('td')[1])
        pitch_search = re.search(r'\d+ pitches',route_type_text)
        if str(type(pitch_search)) == "<class 'NoneType'>":
            num_pitches = 1
        else:
            num_pitches = pitch_search.group(0).split(' ')[0]
        return int(num_pitches)

    df_source['Pitches'] = df_source['Re Mainpage'].progress_apply(get_pitches)
    return df_source

In [7]:
def extract_tick_details(df_source):
    ### From a routes statpage, this analyzes and orders all ticks into a sub-dataframe that can be referenced for further analysis
    
    def get_tick_details(res):

        name = []
        namelink = []
        entrydate = []
        pitches = []
        style = []
        lead_style = []
        comment = []
        soup = BeautifulSoup(res.text, 'html.parser')
        
        # print(soup.select("#route-stats > div.row.pt-main-content > div > h1")) # Tells you which page is being scraped, useful for debugging
        try:
            blocks = list(soup.select("#route-stats > div:nth-child(2) > div:nth-last-child(1)")[0].find_all('tr'))
        except:
            blocks = []
        for x in blocks:
            soup = BeautifulSoup(str(x), 'html.parser')
            entries = soup.find_all('div', attrs={'class': None})
            for entry in entries:
                entrytext = entry.text
                try:
                    name.append(soup.find('a').text.strip())
                except:
                    name.append('')
                    
                try:
                    namelink.append(soup.find('a')['href'].strip())
                except:
                    namelink.append('')
                
                try:
                    date_search = [re.search(r'\w{3}\s\d{1,2},\s\d{4}', entrytext)]
                    entrydate.append([subresult.group(0).strip() if subresult else '' for subresult in date_search][0]) # pulls match text if match object is not none
                except:
                    entrydate.append('')
                
                try:
                    pitches_search = [re.search(r'·([^.]+\s(pitches))', entrytext)] # regex for starting at · and ending at first period only if it includes the word "pitches"
                    pitchesinterm = [subresult.group(0) if subresult else '' for subresult in pitches_search]
                    pitches.append([int(re.search(r'\d+', subresult).group(0).strip()) if subresult else 1 for subresult in pitchesinterm][0]) # take just the digit of the string
                except:
                    pitches.append(1)
                
                try:
                    style_search = [re.search(r"(Solo|TR|Follow|Lead|Send|Attempt|Flash)", entrytext)]
                    style_val = [subresult.group(0).strip() if subresult else '' for subresult in style_search][0] # I have a conditional in the comment search that depends on this so I made it a separate variable
                    style.append(style_val)
                except:
                    style.append('')
                
                try:
                    if style_val != '':
                        lead_style_search = [re.search(r"/([^.]+)", entrytext)]
                        lead_style.append([subresult.group(0)[2:].strip() if subresult else '' for subresult in lead_style_search][0])
                    else:
                        lead_style.append('')
                except:
                    lead_style.append('')
                
                try:
                    if style_val != '': 
                        comment_search = [re.search(r"(Solo|TR|Follow|Lead).*", entrytext)]
                        commentinterm = ([subresult.group(0) if subresult else '' for subresult in comment_search])
                        comment.append([re.search(r"\..*", subresult).group(0)[2:].strip() if subresult else '' for subresult in commentinterm][0])
                    else:
                        comment_search = [re.search(r"·(.*)", entrytext)] # If no style comment then entire phrase is the comment.
                        comment.append([subresult.group(0)[2:].strip() if subresult else '' for subresult in comment_search][0])
                except:
                    comment.append('')
        # print (len(name),len(namelink),len(entrydate),len(pitches),len(style),len(lead_style),len(comment))
        # print (name,namelink,entrydate,pitches,style,lead_style,comment)
        d = pd.DataFrame({'Username' : name, "User Link" : namelink, "Entry Date": entrydate, "Pitches": pitches, "Style": style, "Lead Style": lead_style, "Comment": comment})
        # One last possible error correction, an oomlot injected a "/" into lead style and the regex incidentally detected it
        d.loc[~d['Lead Style'].isin(TICK_OPTIONS), 'Lead Style'] = ''
        return d

    df_source['Route Ticks']=df_source['Re Statpage'].progress_apply(get_tick_details)
    return(df_source)

In [8]:
def tick_analysis(df_source):
    ### Analyzes tick sub dataframe to create meaningful metrics.
    
    def unpack_style(statpage_df, colref):
        nest_list = []
        for row in statpage_df.index:
            styleval = statpage_df[colref][row]
            if styleval in CLEAN_SEND: # clean sends with multiple ticks are assumed to be fell/hung attempts up to that clean send.
                nest_list.append([statpage_df[colref][row]])
                nest_list.append((statpage_df['Pitches'][row]-1)*['Fell/Hung'])
            else:
                nest_list.append(statpage_df['Pitches'][row]*[statpage_df[colref][row]])
        flat_list = [num for sublist in nest_list for num in sublist]
        return flat_list

    def analyze_tick_counts(statpagedf):
        # Get number of ticks and tickers
        num_ticks = len(statpagedf.index)
        num_tickers = statpagedf['Username'].nunique()
        
        # Create tick metrics
        search_counts_init = pd.Series(np.nan, index=TICK_OPTIONS).to_frame()
        tick_type_list = unpack_style(statpagedf, 'Style') + unpack_style(statpagedf, 'Lead Style')
        tick_type_list = list(filter(None, tick_type_list))
        search_counts = pd.Series(tick_type_list).value_counts().dropna().to_frame()
        search_counts = search_counts_init.fillna(search_counts).fillna(0).squeeze()
        lead_ratio = search_counts['Lead']/(search_counts['Follow'] + search_counts['TR'] + search_counts['Lead'])
        os_ratio = (search_counts['Onsight'] + search_counts['Flash']) / (search_counts['Onsight'] + search_counts['Flash'] + search_counts['Fell/Hung'] + search_counts['Redpoint'] + search_counts['Pinkpoint'] + search_counts['Attempt'] + search_counts['Send'])
        return pd.Series([num_ticks, num_tickers, lead_ratio, os_ratio])

    df_source[['Num Ticks', 'Num Tickers', 'Lead Ratio', 'OS Ratio']] = df_source['Route Ticks'].apply(analyze_tick_counts)
    return df_source